In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
price = pd.read_csv('price.csv', sep=';', encoding='windows-1251')

In [3]:
price.head(2)

,Артикул,Категория,"Наименование, харак-ка",количество штук в упаковке,цена за упаковку,Видео,В наличии
0,РС1705,Бенгальские свечи,Бенгальский огонь-170,1000 шт,5165,https://youtu.be/n9cwvZqSq2Q,1
1,РС1710,Бенгальские свечи,Бенгальский огонь 230,360 шт,4570,https://youtu.be/JsNE911bm0E,1


In [4]:
price = price.rename(columns={'Артикул' : '_id', 'Категория' : 'category', \
                              'Наименование, харак-ка' : 'title', \
                              'количество штук в упаковке' : 'value', \
                              'цена за упаковку' : 'price', 'Видео' : 'link', \
                              'В наличии' : 'hide'})

In [5]:
for i in price.columns:
    if i.startswith('Unnamed'):
        price = price.drop(columns=[i])

In [6]:
price = price.dropna()

In [7]:
if type(price.value[0]) == str:
    price.value = price.value.str.rstrip('.').str.rstrip(' шт').astype('int')

In [8]:
if price.price.dtypes == 'object':
    price.price = price.price.str.replace(',', '.').astype('float')
elif price.price.dtypes == 'int64':
    price.price = price.price.astype('float')

In [9]:
price.category = price.category.str.rstrip(' ').str.rstrip('.')

In [10]:
price.title = price.title.str.replace('\n', ' ')

In [11]:
for i in range(len(price.index)):
    if price.loc[i, 'hide'] == 0:
        price.loc[i, 'hide'] = True
    else:
        price.loc[i, 'hide'] = False

In [12]:
price = price.fillna('')

In [13]:
price.title = price._id + ' ' + price.title

In [14]:
price.head()

,_id,category,title,value,price,link,hide
0,РС1705,Бенгальские свечи,РС1705 Бенгальский огонь-170,1000,5165.0,https://youtu.be/n9cwvZqSq2Q,False
1,РС1710,Бенгальские свечи,РС1710 Бенгальский огонь 230,360,4570.0,https://youtu.be/JsNE911bm0E,False
2,РС1720,Бенгальские свечи,РС1720 Бенгальский огонь 300,3,100.0,https://youtu.be/oOArRIsdteE,False
3,РС1730,Бенгальские свечи,РС1730 Бенгальский огонь 300 УЛЬТРА,4,155.0,https://youtu.be/gVy3SthKDnw,False
4,РС1755,Бенгальские свечи,РС1755 Цветной бенгальский огонь 300,300,4035.0,https://youtu.be/OqZV70OSlPk,False


In [15]:
price_columns = price.columns

In [16]:
# add categories for json
categories = [{"title": "Все", "value": "all"}]
used_categories = []
for i in price.index:
    category = {}
    if price.loc[i, 'category'] not in used_categories:
        category["title"] = price.loc[i,'category']
        category["value"] = price.loc[i,'category'] #FIX!
        used_categories.append(price.loc[i,'category'])
        categories.append(category)

for i in categories:
    if i['title'] == 'Хлопушки. Начинка, длина':
        i['title'] = 'Хлопушки'
    elif i['title'] == 'Пневмохлопушки. Начинка, длина':
        i['title'] = 'Пневмохлопушки'
    elif i['title'] == 'Римские свечи (калибр х колич.выстрелов)':
        i['title'] = 'Римские свечи'
    elif i['title'] == 'Одиночные салюты, фестивальные шары. (калибр " Х колич шаров)':
        i['title'] = 'Одиночные салюты, фестивальные шары'
    elif i['title'] == 'Батареи салютов  (калибр Х колич. стволов)':
        i['title'] = 'Батареи салютов'

In [17]:
# add items for json
items = []
for i in range(len(price.index)):
    item = {}
    price_row = price.loc[i,:].to_list()
    for j in range(len(price_columns)):
        item[price_columns[j]] = price_row[j]
    items.append(item)

In [18]:
items_to_json = {}
items_to_json['categories'] = categories
items_to_json['items'] = items

In [19]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.bool_):
            return bool(obj)
        return super(NpEncoder, self).default(obj)
    
with open('price.json', 'w') as json_file:
    json.dump(items_to_json, json_file, indent=1, ensure_ascii=False , cls=NpEncoder)